In [ ]:
%pip install --upgrade langchain langchain-experimental langchain-openai python-dotenv pyvis

In [1]:
import json, re, requests

In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv(override=True)
# Get API key from environment variable 
api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [111]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from typing import List, Dict, Tuple, Optional, Iterable, TypedDict
from pydantic import BaseModel, Field

from langchain_community.document_transformers import Html2TextTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

from bs4 import BeautifulSoup


### LLM Graph Transformer
Using GPT-4o in all examples.

In [125]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
graph_transformer = LLMGraphTransformer(llm=llm)

In [ ]:

class AgentState(TypedDict):
    urls_to_scrape: List[str]
    scraped_urls: List[str]
    
    accumulated_text: str 
    extracted_triples: List[dict] 

In [71]:
class KnowledgeTriple(BaseModel):
    """A structured relationship or fact suitable for a knowledge graph."""
    subject: str = Field(description="The main concept (entity) in the relationship. E.g., 'Hormone Therapy'")
    relationship: str = Field(description="The verb or phrase linking the subject and object. E.g., 'treats', 'has_symptoms', 'increases_risk'")
    object: str = Field(description="The target concept (entity) in the relationship. E.g., 'Hot Flashes'")

# 最终的输出模型
class KnowledgeGraphOutput(BaseModel):
    """The complete structured output for knowledge graph construction."""
    menopause_focus: str = Field(description="A 3-5 word summary of the main focus of the text regarding menopause.")
    extracted_triples: List[KnowledgeTriple] = Field(description="A list of structured triples extracted from the text.")

### Search Agent 
Find relevant web sources

In [68]:
# researcher_agent_tavily.py
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, SystemMessage
# --- 1. Define the Tavily Search Tool ---
# max_results=7: Request 7 results to give the LLM more options for authoritative sources.
# search_depth="advanced": Tavily will use a deeper search to find high-quality content.
tavily_search_tool = TavilySearchResults(
    max_results=15,
    search_depth="advanced",
    name="tavily_search_tool",
    description=(
        "A highly optimized search engine for AI agents. Use this to find relevant "
        "and authoritative URLs (e.g., .gov, .edu, .int, reputable health organizations like WHO, CDC, NIH, UptoDate, or medical institutions like Stanford, Havard, Mayo Clinic, Cleveland Clinic) "
        "for knowledge graph construction. Input should be a concise search query."
    ),
    tavily_api_key=tavily_api_key
)


search_agent_system_prompt = """
    You are an expert researcher specializing in women health and medical topics. Your goal is to find the most 
    authoritative and relevant URLs related to 'menopause and women's health' for building a knowledge graph.

    **CRITERIA for selecting URLs:**
    1. **Authority:** Must be highly authoritative (e.g., government sites like .gov, .edu, WHO, NIH, CDC, and major reputable medical journals/clinics like Mayo Clinic, JHU), exclude PubMed articles.
    2. **Relevance:** Must directly relate to core menopause concepts (symptoms, causes, treatments, health advice).

    You **MUST** use the 'tavily_search_tool' to find a list of potential URLs. 
    After searching, carefully analyze the JSON search results (snippets) and output ONLY a **clean, comma-separated list of the authoritative URLs** that are most relevant to the knowledge graph construction. Do not output any prose, thoughts, or formatting other than the URL list.
    """

# --- 2. Create the Researcher Agent (ReAct) ---
def tavily_researcher_agent():
    """
    Sets up a ReAct Agent capable of using the Tavily Search Tool to find authoritative URLs.
    """
    tools = [tavily_search_tool]   
    # 2.3 Create the ReAct Agent and Executor
    agent = create_agent(
        model=llm,
        tools=tools,
        system_prompt=search_agent_system_prompt,
    )    
    search_query = "Find 20 authoritative websites for comprehensive menopause knowledge and relevant recommendations"
    result = agent.invoke({"input": search_query})
    return result['messages'][-1].content.split(",")

print(f"--- Researcher Agent Start: {search_query} ---\n")
urls = tavily_researcher_agent()
# result = researcher_agent.invoke({"input": search_query})

# The agent will execute the search and its reasoning loop
# # We expect the final output to be a string of comma-separated URLs
# inputs = {"input": search_query}
# for chunk in researcher_agent.stream(inputs, stream_mode="updates"):
#     print(chunk)


# The result structure from AgentExecutor needs robust parsing, 
# but for a simplified example, we rely on the prompt instructing a clean output.
# result_str = result.get("output", "")
# authoritative_urls = [url.strip() for url in result_str.split(',') if url.strip() and url.startswith('http')]

# print("\n==============================================")
# print(f"Final Found Authoritative URLs ({len(authoritative_urls)}):")
# for url in authoritative_urls:
#     print(f"- {url}")
# print("==============================================")

# This list of URLs is then fed into your Scrape Agent loop.

# Uncomment to run the example:
# if __name__ == "__main__":
#     run_tavily_researcher()

--- Researcher Agent Start: authoritative websites for comprehensive menopause knowledge and relevant advice ---



In [66]:
urls

['https://www.nia.nih.gov/health/menopause/what-menopause',
 ' https://www.cdc.gov/womens-health/features/menopause-womens-health-and-work.html',
 ' https://www.nlm.nih.gov/medlineplus/menopause.html',
 ' https://www.nichd.nih.gov/health/topics/menopause/conditioninfo/symptoms',
 ' https://www.nccih.nih.gov/health/menopausal-symptoms-in-depth',
 ' https://www.who.int/news-room/fact-sheets/detail/menopause',
 ' https://go.nih.gov/mKSWdYS',
 ' https://www.nichd.nih.gov/health/topics/menopause/conditioninfo/treatments',
 ' https://www.ncbi.nlm.nih.gov/books/NBK507826/',
 ' https://www.ncbi.nlm.nih.gov/books/NBK279309/']

In [69]:
urls

['https://www.nia.nih.gov/health/menopause/what-menopause',
 ' https://www.cdc.gov/womens-health/features/menopause-womens-health-and-work.html',
 ' https://www.nichd.nih.gov/health/topics/menopause/conditioninfo/symptoms',
 ' https://www.nccih.nih.gov/health/menopausal-symptoms-in-depth',
 ' https://www.who.int/news-room/fact-sheets/detail/menopause',
 ' https://go.nih.gov/mKSWdYS',
 ' https://www.nichd.nih.gov/health/topics/menopause/conditioninfo/treatments',
 ' https://www.ncbi.nlm.nih.gov/books/NBK507826/',
 ' https://www.ncbi.nlm.nih.gov/books/NBK279309/',
 ' https://www.nlm.nih.gov/medlineplus/menopause.html']

### Scrape Data from Web Sources

In [80]:
# scrape_agent.py
def web_scrape_tool(url: str) -> str:
    """
    Scrapes the text content from a given URL.

    Args:
        url: The URL of the web page to scrape.

    Returns:
        The clean, readable text content of the page.
    """
    # 1. Fetch the content from the URL
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

    # 2. Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # 3. Extract all readable text from the page
    page_text = soup.get_text(separator=' ', strip=True)

    return page_text

In [ ]:
# not using it !!!
scrape_agent_system_prompt = """
    You are an expert web scraping processor. Your goal is to methodically scrape the content from a list of URLs 
    provided in the human input.

    **TASK FLOW:**
    1. The human input will be a list of string of URLs to scrape.
    2. You **MUST** call the 'web_scrape_tool' sequentially for **EACH URL** in the list.
    3. The input to the 'web_scrape_tool' must be **one URL at a time**.
    4. After the tool returns the text content and status, you should move to the next URL.

    **FINAL OUTPUT:**
    After scraping ALL provided URLs, summarize the *main topic* and *authority* of the content gathered from the successful scrapes. 
    Do **NOT** output the full scraped text or the tool logs. 
    Your output should be a single, concise paragraph summarizing the collected content's key themes.
    """

def setup_web_scraper_agent(llm: ChatOpenAI, web_scraper_tool: tool):
    """
    Sets up an Agent capable of using the Web Scraper Tool to gather content from multiple URLs.
    """
    tools = [web_scraper_tool]
    
    # 使用纯字符串 system_prompt
    agent = create_agent(
        model=llm,
        tools=tools,
        system_prompt=scrape_agent_system_prompt,
    )    
    return agent

In [129]:
concept_extractor_system_prompt = """
    You are an expert medical knowledge graph constructor specializing in menopause and women's health. 
    Your task is to analyze the provided web page text and extract ALL relevant concepts and relationships 
    into a structured list of knowledge triples.

    **REQUIRED CONCEPTS (Subjects/Objects):**
    Map synonyms to one concept, for example: "hot flashes"/"hot flushes" to the same Symptom.
    Keep names concise (e.g., 'hot flashes', 'soy foods', 'Mediterranean diet'). 
    You MUST prioritize concepts related to the following categories:
    - **Menopause Core:** (e.g., Menopause, Perimenopause, Postmenopause)
    - **Symptoms:** (e.g., Hot flashes, Mood swings, Insomnia, Vaginal atrophy)
    - **Treatments/Interventions:** (e.g., Hormone therapy (HT), Estrogen therapy, Cognitive Behavioral Therapy (CBT), Supplements)
    - **Lifestyle/Factors:** (e.g., Diet, Supplement, Exercise, Sleep, Stress management, Smoking, etc.)
    - **Outcomes/Risks:** (e.g., Osteoporosis, Cardiovascular disease risk, Bone density)
    - **Populations:** (e.g., Postmenopausal women, Early menopause patients)

    
    **REQUIRED RELATIONSHIPS:**
    - Must clearly define the connection (e.g., 'causes', 'treats', 'mitigates', 'increases_risk_of', 'is_a_type_of', 'affects', etc.).
    - Map synonyms to one relationship, for example: "a common symptom of"/"are a symptom of"/"is a symptom of" to the same relationship "symptom of".
    
    You MUST adhere strictly to the provided output JSON schema.
    """

def concept_extractor():
    """
    Sets up a non-tool-using Agent to extract structured knowledge triples from text.
    """
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", concept_extractor_system_prompt),
        ("human", "Analyze the following text and extract all knowledge triples: \n\nTEXT:\n{input}"),
    ])
    extractor = LLMGraphTransformer(llm=llm, prompt=prompt_template)
    # kg_chain = prompt_template | llm.with_structured_output(KnowledgeGraphOutput)
    
    return extractor



sample_scraped_text = (
    "Menopause, often starting with perimenopause, commonly presents with severe hot flashes and sleep disruption. "
    "Hormone therapy is the most effective treatment for vasomotor symptoms, but it increases the risk of stroke in some women. "
    "Weight-bearing exercise is recommended to improve bone density and mitigate the long-term risk of osteoporosis."
)
print("--- Knowledge Graph Extraction Start ---")
test_text = web_scrape_tool("https://australianprescriber.tg.org.au/articles/management-of-menopause.html")
extractor = concept_extractor()
single_doc1 = [Document(page_content=test_text, metadata={"source": "manual_string"})]
graph_doc1 = extractor.convert_to_graph_documents(single_doc1)
# print(f"Nodes:{graph_documents[0].nodes}")
# print(f"Relationships:{graph_documents[0].relationships}")

--- Knowledge Graph Extraction Start ---


In [ ]:
# uptodate
test_text2 = web_scrape_tool("https://www.uptodate.com/contents/menopause-clinical-features-and-diagnosis")
single_doc2 = [Document(page_content=test_text, metadata={"source": "manual_string"})]
graph_doc2 = extractor.convert_to_graph_documents(single_doc2)


In [139]:
len(graph_doc2[0].nodes)
len(graph_doc1[0].nodes)

30

In [141]:
# combine the article & uptodate
graph_documents = []
# graph_documents.extend(graph_doc1)
graph_documents.extend(graph_doc2)
print(f"Nodes:{len(graph_documents[0].nodes)}")
visualize_graph(graph_documents)

Nodes:32
Graph saved to /Users/ml5128/Documents/BINFG4003_SymbolicAI/Project/menopause_knowledge_graph_by_LLM/knowledge_graph.html


AACT Text, need chunking

In [115]:
raw_aact=""
with open('structured_for_llm.txt', 'r', encoding='utf-8') as file:
    raw_aact = file.read()
    #print(content)
aact_data = Document(page_content =raw_aact)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200, # 允许块之间有重叠以保留上下文
    length_function=len,
    is_separator_regex=False,
)
chunked_aact = text_splitter.split_documents([aact_data])
print(f"AACT data split into {len(chunked_aact)} chunks.")

AACT data split into 685 chunks.


In [118]:
graph_aact = extractor.convert_to_graph_documents(chunked_aact[:30])
# all_graph_documents.extend(graph_documents_from_url)
print(f"Extracted {len(graph_aact)} Graph Documents from this URL.")

Extracted 30 Graph Documents from this URL.


In [120]:
graph_documents.extend(graph_aact)

In [ ]:
# urls (not running for now)
graph_documents = []
for i in urls:
    if "ncbi" not in i:
        print("=======")
        try:
            print(f"extracting url: {i}")
            raw_text = web_scrape_tool(i)
            #extracted_data = kg_extractor_chain.invoke({"input": raw_text})

            documents = [Document(page_content=raw_text, metadata={"source": "manual_string"})]
            single_graph_doc = extractor.convert_to_graph_documents(documents)
            graph_documents.extend(single_graph_doc)
            #print(f"Menopause Focus: {extracted_data.menopause_focus}")
            # print("\nExtracted Triples:")
            # for triple in extracted_data.extracted_triples:
            #     print(f"({triple.subject}) --[{triple.relationship}]--> ({triple.object})")
        except Exception as e:
            print(f"unable to scrape, reason: {e}")
        print()
print(len(graph_documents))

In [ ]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")


In [61]:
def get_pmc_article_data(url: str) -> str:
    """
    Retrieves abstract, full text link, and citation info for a PMC article 
    by using the PMC Open Access Web Service, which is the official method 
    to programmatically access this site's content.

    Args:
        url: The full URL of the PMC article (e.g., '.../PMC10665088/').

    Returns:
        A formatted string containing the article's title, abstract, and links, 
        or an error message if the ID or API call fails.
    """
    pmc_id = url.split('/')[-2] # 假设倒数第二个元素是ID
    if not pmc_id.startswith('PMC'):
        pmc_id = url.split('/')[-1] # 如果是末尾
        if not pmc_id.startswith('PMC'):
            return f"API_ERROR: Could not find valid PMC ID in URL: {url}"
    base_api_url = "https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi"
    params = {
        "id": pmc_id,
        "format": "json", # 请求 JSON 格式的元数据
    }

    try:
        response = requests.get(base_api_url, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()
    
    except requests.exceptions.RequestException as e:
        return f"API_FETCH_ERROR: Failed to connect to PMC API for {pmc_id}. Reason: {e}"
    except json.JSONDecodeError:
        return f"API_FETCH_ERROR: Received non-JSON response from API for {pmc_id}."

    # 4. 解析和格式化数据
    
    # 检查 API 是否返回了文章数据
    record = data.get('records', [])
    if not record:
        return f"API_ERROR: Article {pmc_id} not found in PMC records or is not open access."
        
    article = record[0]
    
    # 提取关键信息
    title = article.get('title', 'N/A')
    pub_date = article.get('pubDate', 'N/A')
    
    # 获取全文下载链接（通常是XML或PDF）
    full_text_link = "N/A"
    if 'link' in article:
        full_text_link = article['link'].get('href', 'N/A')
        
    # **注意：API通常不直接返回抽象的文本内容。** # **要获取摘要/文本，需要通过另一个API，或者解析获取到的全文链接 (full_text_link)。**
    
    # 为了简化，我们只返回基础信息和链接：
    formatted_output = (
        f"--- PMC ARTICLE METADATA ({pmc_id}) ---\n"
        f"TITLE: {title}\n"
        f"PUB_DATE: {pub_date}\n"
        f"PMC_LINK: {url}\n"
        f"FULL_TEXT_XML_LINK: {full_text_link}\n"
        f"STATUS: SUCCESS\n"
    )
    
    return formatted_output

# --- 示例调用 ---
article_url = 'https://pmc.ncbi.nlm.nih.gov/articles/PMC10665088/'
result_data = get_pmc_article_data(article_url)
print(result_data)

API_FETCH_ERROR: Failed to connect to PMC API for PMC10665088. Reason: Expecting value: line 1 column 1 (char 0)


In [ ]:
# scrape the webpage and it's following urls
def scrape_page_and_extract_links(url: str, base_domain: str) -> tuple[str, List[str]]:
    """
    Scrapes the text content from a given URL and it's relevant URL on the website.
    """

    response = requests.get(url, timeout=10)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # 提取所有文本内容
    page_text = soup.get_text(separator=' ', strip=True)
    print(page_text)
    print("========")
    # 提取所有链接
    new_links = []
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        # 简单的链接过滤：确保是完整的 HTTP/HTTPS 链接，且属于目标域名
        if href.startswith('http') and base_domain in href:
            new_links.append(href)
        elif href.startswith('/') and not href.startswith('//'):
            # 处理相对路径链接
            full_url = requests.compat.urljoin(url, href)
            if base_domain in full_url:
                new_links.append(full_url)
    
    return page_text, new_links

# Sample code downloaded from tutorials

In [40]:
URL = "https://zoe.com/learn/foods-that-ease-hot-flashes"

# —— 1) 抓取网页并转纯文本 ——
docs = WebBaseLoader(URL).load()
plain = Html2TextTransformer().transform_documents(docs)

splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
chunks = splitter.split_documents(plain)


In [ ]:
text = plain[0].page_content
index = text.find("https")
trimmed_text = text[:index]
trimmed_text

In [ ]:
SCHEMA_HINT = """
You are extracting a menopause knowledge graph from text.
You need to identify all concepts such as symptoms, food items, bioactive chemicals, and their relationships.

[RULES]
- Map synonyms to one concept, for example: "hot flashes"/"hot flushes" to the same Symptom.
- Keep names concise (e.g., 'hot flashes', 'soy foods', 'Mediterranean diet'). 

Return triples only; no summaries.
"""

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME", "gpt-4o"), temperature=0)

graph_prompt = ChatPromptTemplate.from_messages([
("system", SCHEMA_HINT + "Return triples only; do not summarize."),
("human", "Extract menopause knowledge graph triples from given text. Return only graph objects."),
])

extractor = LLMGraphTransformer(llm=llm, prompt=graph_prompt)

In [47]:
documents = [Document(page_content=text, metadata={"source": "manual_string"})]
graph_documents = extractor.convert_to_graph_documents(documents)

In [48]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Hot Flashes', type='Symptom', properties={}), Node(id='Soy Foods', type='Food', properties={}), Node(id='Isoflavones', type='Bioactive chemical', properties={}), Node(id='Mediterranean Diet', type='Food', properties={}), Node(id='Fruits', type='Food', properties={}), Node(id='Vegetables', type='Food', properties={}), Node(id='Whole Grains', type='Food', properties={}), Node(id='Nuts', type='Food', properties={}), Node(id='Seeds', type='Food', properties={})]
Relationships:[Relationship(source=Node(id='Soy Foods', type='Food', properties={}), target=Node(id='Isoflavones', type='Bioactive chemical', properties={}), type='CONTAIN', properties={}), Relationship(source=Node(id='Isoflavones', type='Bioactive chemical', properties={}), target=Node(id='Hot Flashes', type='Symptom', properties={}), type='REDUCE', properties={}), Relationship(source=Node(id='Mediterranean Diet', type='Food', properties={}), target=Node(id='Fruits', type='Food', properties={}), type='INCLUDE', pro

### Extract graph data

In [94]:
text = """
Albert Einstein[a] (14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is best known for developing the theory of relativity. Einstein also made important contributions to quantum mechanics.[1][5] His mass–energy equivalence formula E = mc2, which arises from special relativity, has been called "the world's most famous equation".[6] He received the 1921 Nobel Prize in Physics for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect.[7]

Born in the German Empire, Einstein moved to Switzerland in 1895, forsaking his German citizenship (as a subject of the Kingdom of Württemberg)[note 1] the following year. In 1897, at the age of seventeen, he enrolled in the mathematics and physics teaching diploma program at the Swiss federal polytechnic school in Zurich, graduating in 1900. He acquired Swiss citizenship a year later, which he kept for the rest of his life, and afterwards secured a permanent position at the Swiss Patent Office in Bern. In 1905, he submitted a successful PhD dissertation to the University of Zurich. In 1914, he moved to Berlin to join the Prussian Academy of Sciences and the Humboldt University of Berlin, becoming director of the Kaiser Wilhelm Institute for Physics in 1917; he also became a German citizen again, this time as a subject of the Kingdom of Prussia.[note 1] In 1933, while Einstein was visiting the United States, Adolf Hitler came to power in Germany. Horrified by the Nazi persecution of his fellow Jews,[8] he decided to remain in the US, and was granted American citizenship in 1940.[9] On the eve of World War II, he endorsed a letter to President Franklin D. Roosevelt alerting him to the potential German nuclear weapons program and recommending that the US begin similar research.

In 1905, sometimes described as his annus mirabilis (miracle year), he published four groundbreaking papers.[10] In them, he outlined a theory of the photoelectric effect, explained Brownian motion, introduced his special theory of relativity, and demonstrated that if the special theory is correct, mass and energy are equivalent to each other. In 1915, he proposed a general theory of relativity that extended his system of mechanics to incorporate gravitation. A cosmological paper that he published the following year laid out the implications of general relativity for the modeling of the structure and evolution of the universe as a whole.[11][12] In 1917, Einstein wrote a paper which introduced the concepts of spontaneous emission and stimulated emission, the latter of which is the core mechanism behind the laser and maser, and which contained a trove of information that would be beneficial to developments in physics later on, such as quantum electrodynamics and quantum optics.[13]

In the middle part of his career, Einstein made important contributions to statistical mechanics and quantum theory. Especially notable was his work on the quantum physics of radiation, in which light consists of particles, subsequently called photons. With physicist Satyendra Nath Bose, he laid the groundwork for Bose–Einstein statistics. For much of the last phase of his academic life, Einstein worked on two endeavors that ultimately proved unsuccessful. First, he advocated against quantum theory's introduction of fundamental randomness into science's picture of the world, objecting that God does not play dice.[14] Second, he attempted to devise a unified field theory by generalizing his geometric theory of gravitation to include electromagnetism. As a result, he became increasingly isolated from mainstream modern physics.
"""

In [7]:
documents = [Document(page_content=text)]
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

# Visualize graph

In [121]:
from pyvis.network import Network

def visualize_graph(graph_documents):

    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")
    
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}
    
    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)
        
    output_file = "knowledge_graph.html"
    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")
        
# Run the function
visualize_graph(graph_documents)

Graph saved to /Users/ml5128/Documents/BINFG4003_SymbolicAI/Project/menopause_knowledge_graph_by_LLM/knowledge_graph.html


In [122]:
visualize_graph(graph_aact)

Graph saved to /Users/ml5128/Documents/BINFG4003_SymbolicAI/Project/menopause_knowledge_graph_by_LLM/knowledge_graph.html


### Extract specific types of nodes

In [40]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
graph_transformer_nodes_defined = LLMGraphTransformer(llm=llm, allowed_nodes=allowed_nodes)
graph_documents_nodes_defined = await graph_transformer_nodes_defined.aconvert_to_graph_documents(documents)

In [ ]:
print(f"Nodes:{graph_documents_nodes_defined[0].nodes}")
print(f"Relationships:{graph_documents_nodes_defined[0].relationships}")

### Extract specific types of relationships

In [42]:
allowed_nodes = ["Person", "Organization", "Location", "Award", "ResearchField"]
allowed_relationships = [
    ("Person", "WORKS_AT", "Organization"),
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField")
]
graph_transformer_rel_defined = LLMGraphTransformer(
  llm=llm,
  allowed_nodes=allowed_nodes,
  allowed_relationships=allowed_relationships
)
graph_documents_rel_defined = await graph_transformer_rel_defined.aconvert_to_graph_documents(documents)

In [43]:
# Visualize graph
visualize_graph(graph_documents_rel_defined)

Graph saved to /Users/thuvu/Documents/vlogging/Research/knowledge_graph_app/knowledge_graph.html
